In [5]:
import spacy
from spacy.matcher import Matcher

In [4]:
nlp = spacy.load("nb_core_news_lg")

In [6]:
matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": "marius"}, {"LOWER": "dioli"}]

In [7]:
tx = "Hei, mitt navn er Marius Dioli. Eg er nysgjerrig på kor mange poeng eg har."

In [8]:
def on_match(matcher, doc, id, matches):
      print('Matched!', matches)

In [9]:
matcher.add("MD matcher", on_match, pattern)

In [10]:
doc = nlp(tx)

In [11]:
matches = matcher(doc)

Matched! [(10180913855843715433, 5, 7)]


In [12]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

10180913855843715433 MD matcher 5 7 Marius Dioli
